# Submitting calculations with AiiDA
#### Create a structure, kpoints, and input parameters and submit a Quantum ESPRESSO (PW) calculation

In [1]:
from aiida.backends.utils import load_dbenv, is_dbenv_loaded
if not is_dbenv_loaded():
    load_dbenv()

from aiida.orm.utils import load_node
from aiida.work.run import run
from aiida.orm.code import Code
from aiida.orm.data.base import Bool, Str
from aiida.orm.data.parameter import ParameterData
from aiida.orm.data.array.kpoints import KpointsData
from aiida.orm.data.structure import StructureData
from ase.lattice.spacegroup import crystal

#### Create a diamond cubic crystal structure

In [2]:
# Define a structure, employing the Atomic Structure Environment library
alat = 3.568
ase_definition = crystal('C', [(0,0,0)], spacegroup=227, cellpar=[alat, alat, alat, 90, 90, 90], primitive_cell=True)*2
structure = StructureData(ase=ase_definition)
structure.store()

<StructureData: uuid: 43cd5aa5-a4b6-46f7-bb76-80b364d287a6 (pk: 5037)>

#### Create the _k_-points mesh

In [3]:
# Define the k-points mesh
kpoints = KpointsData()
kpoints_mesh = [1, 1, 1]
kpoints.set_kpoints_mesh(kpoints_mesh)
kpoints.store()

<KpointsData: uuid: 4e187b06-b829-488e-adeb-a4dcb027c8e6 (pk: 5038)>

#### Define the calculation input parameters

In [4]:
# Define the pseudo potential family and input parameters for pw.x
pseudo_family = 'SSSP'
parameters = {
    'CONTROL': {
        "calculation": "scf",
        'tstress': True,
    },
    'SYSTEM': {
        'ecutwfc': 40.,
        'ecutrho': 320.,
    },
    'ELECTRONS': {
        'conv_thr': 1.e-10,
    }
}

#### See the available codes and select one of them

In [5]:
!verdi code list

# List of configured codes:
# (use 'verdi code show CODEID' to see the details)
* pk 4681 - pw-5.1@localhost


#### Tell AiiDA which code to use

In [6]:
codename = 'pw-5.1'
code = Code.get_from_string(codename)

#### Create a new calculation from the code and set the inputs

In [7]:
calc = code.new_calc()
calc.use_structure(structure)
calc.use_pseudos_from_family(pseudo_family)
calc.use_parameters(ParameterData(dict=parameters))
calc.use_kpoints(kpoints)
calc.set_resources({"num_machines": 1})
calc.set_max_wallclock_seconds(30*60)

#### Store the calculation and submit it

In [8]:
calc.store_all()
calc.submit()
print "Submitted a {} with pk<{}>".format(calc.__class__.__name__, calc.pk)

Submitted a PwCalculation with pk<5040>


#### Check the status of the calculation

In [9]:
!verdi calculation list

# Last daemon state_updater check: 0h:00m:24s ago (at 17:51:59 on 2017-04-20)
  PK  State     Creation      Sched. state  Computer    Type
----  --------  ----------  --------------  ----------  ------------------
5040  TOSUBMIT  4s ago                      localhost   quantumespresso.pw

Total results: 1



#### Monitor its status until completion

In [10]:
import time
from IPython.display import clear_output

def check_calculation_status(calc):
    while not calc.has_finished():
        clear_output()
        !verdi calculation list
        time.sleep(2)

    clear_output()
    !verdi calculation list $calc.pk

In [11]:
check_calculation_status(calc)

# Last daemon state_updater check: 0h:00m:06s ago (at 17:53:29 on 2017-04-20)
  PK  State     Creation    Sched. state    Computer    Type
----  --------  ----------  --------------  ----------  ------------------
5040  FINISHED  1m ago      DONE            localhost   quantumespresso.pw

Total results: 1



#### Load the calculation node and check if it finished correctly

In [12]:
calculation = load_node(calc.pk)
calculation.has_finished_ok()

True

#### We can easily show all the attached outputs

In [13]:
calculation.get_outputs_dict()

{u'output_array': <ArrayData: uuid: ac518c1b-3154-4a87-bc31-615ae58f3d84 (pk: 5045)>,
 u'output_array_5045': <ArrayData: uuid: ac518c1b-3154-4a87-bc31-615ae58f3d84 (pk: 5045)>,
 u'output_kpoints': <KpointsData: uuid: f67e7bda-c025-46ee-94bf-b11cf8ff3c27 (pk: 5043)>,
 u'output_kpoints_5043': <KpointsData: uuid: f67e7bda-c025-46ee-94bf-b11cf8ff3c27 (pk: 5043)>,
 u'output_parameters': <ParameterData: uuid: bb19f108-7d18-4186-8f6f-0347a0724e41 (pk: 5044)>,
 u'output_parameters_5044': <ParameterData: uuid: bb19f108-7d18-4186-8f6f-0347a0724e41 (pk: 5044)>,
 u'remote_folder': <RemoteData: uuid: 042c2163-0896-4a06-9361-aec2dcda6f78 (pk: 5041)>,
 u'remote_folder_5041': <RemoteData: uuid: 042c2163-0896-4a06-9361-aec2dcda6f78 (pk: 5041)>,
 u'retrieved': <FolderData: uuid: 11a8b166-583d-4c39-8bfe-2fac06f1f98d (pk: 5042)>,
 u'retrieved_5042': <FolderData: uuid: 11a8b166-583d-4c39-8bfe-2fac06f1f98d (pk: 5042)>}

#### There are convenient methods to directly access the results

In [14]:
print "Total energy = {} {}".format(calculation.res.energy, calculation.res.energy_units)

Total energy = -2471.82841453 eV
